<a href="https://colab.research.google.com/github/AdamJensen93/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [4]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

In [6]:
# Create filtered DataFrame
vine_filtered = vine_df.filter('total_votes>=20')

vine_filtered.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R3EM8C9CKA6GPK|          2|           41|        115|   N|                N|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|     

In [8]:
# Create second filtered DataFrame
vine_votes = vine_filtered.filter((vine_filtered.helpful_votes / vine_filtered.total_votes) >= 0.5)

vine_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [9]:
# Create DataFrame with all the rows where a review was written as part of the Vine program (paid)
vine_paid = vine_votes.filter(vine_votes['vine'] == 'Y')

vine_paid.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37L6YZDX0P3J9|          1|           28|         35|   Y|                N|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|                N|
|R10XRAOBFFSTL2|          5|           44|         47|   Y|                N|
| R1X8F3R1K1SF5|          2|           30|         31|   Y|                N|
|R326QV66CKBB43|          4|          127|        139|   Y|                N|
| RGXLFTR31J561|          5|           29|         34|   Y|                N|
|R2ZS3WCJYUDQH0|          3|          101|        115|   Y|                N|
|R31771VBPRDVJS|          4|           75|         83|   Y|                N|
| R8O8KM73GO190|          5|           21|         28|   Y|                N|
| RPU1GURA1NOT0|          5|           33|         35|   Y|     

In [10]:
# Create DataFrame with all the rows where the review was not part of the Vine program (unpaid)
vine_unpaid = vine_votes.filter(vine_votes['vine'] == 'N')

vine_unpaid.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [11]:
# Total number of reviews (Paid)
total_paid_reviews = vine_paid.count()

total_paid_reviews

607

In [12]:
# Total number of reviews (Unpaid)
total_unpaid_reviews = vine_unpaid.count()

total_unpaid_reviews

50522

In [13]:
# Total number of 5-star reviews (Paid)
five_star_paid = vine_paid.filter("star_rating=5").count()

five_star_paid

257

In [14]:
# Total number of 5-star reviews (Unpaid)
five_star_unpaid = vine_unpaid.filter("star_rating=5").count()

five_star_unpaid

25220

In [15]:
# Percentage of 5-star reviews (Paid)
five_star_paid_percent = (five_star_paid / total_paid_reviews) *100

five_star_paid_percent

42.33937397034596

In [16]:
# Percentage of 5-star reviews (Unpaid)
five_star_unpaid_percent = (five_star_unpaid / total_unpaid_reviews) *100

five_star_unpaid_percent

49.91884723486798